Um die folgenden Notebooks auszuführen, müssen Sie, falls noch nicht geschehen, ein Modell bereitstellen, das `text-embedding-ada-002` als Basismodell verwendet, und den Bereitstellungsnamen in der .env-Datei als `AZURE_OPENAI_EMBEDDINGS_ENDPOINT` festlegen.


In [ ]:
import os
import pandas as pd
import numpy as np
from openai import AzureOpenAI
from dotenv import load_dotenv

load_dotenv()

client = AzureOpenAI(
  api_key=os.environ['AZURE_OPENAI_API_KEY'],  # this is also the default, it can be omitted
  api_version = "2023-05-15"
  )

model = os.environ['AZURE_OPENAI_EMBEDDINGS_DEPLOYMENT']

SIMILARITIES_RESULTS_THRESHOLD = 0.75
DATASET_NAME = "../embedding_index_3m.json"

Als Nächstes laden wir den Embedding-Index in ein Pandas Dataframe. Der Embedding-Index ist in einer JSON-Datei namens `embedding_index_3m.json` gespeichert. Der Embedding-Index enthält die Embeddings für jedes der YouTube-Transkripte bis Ende Oktober 2023.


In [ ]:
def load_dataset(source: str) -> pd.core.frame.DataFrame:
    # Load the video session index
    pd_vectors = pd.read_json(source)
    return pd_vectors.drop(columns=["text"], errors="ignore").fillna("")

Als Nächstes erstellen wir eine Funktion namens `get_videos`, die den Embedding-Index nach der Anfrage durchsucht. Die Funktion gibt die 5 Videos zurück, die der Anfrage am ähnlichsten sind. Die Funktion arbeitet wie folgt:

1. Zuerst wird eine Kopie des Embedding-Index erstellt.
2. Anschließend wird das Embedding für die Anfrage mithilfe der OpenAI Embedding API berechnet.
3. Dann wird im Embedding-Index eine neue Spalte namens `similarity` erstellt. Die Spalte `similarity` enthält die Kosinus-Ähnlichkeit zwischen dem Embedding der Anfrage und dem Embedding jedes Videosegments.
4. Danach wird der Embedding-Index anhand der Spalte `similarity` gefiltert. Es werden nur Videos berücksichtigt, deren Kosinus-Ähnlichkeit mindestens 0,75 beträgt.
5. Abschließend wird der Embedding-Index nach der Spalte `similarity` sortiert und die 5 besten Videos werden zurückgegeben.


In [ ]:
def cosine_similarity(a, b):
    if len(a) > len(b):
        b = np.pad(b, (0, len(a) - len(b)), 'constant')
    elif len(b) > len(a):
        a = np.pad(a, (0, len(b) - len(a)), 'constant')
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

def get_videos(
    query: str, dataset: pd.core.frame.DataFrame, rows: int
) -> pd.core.frame.DataFrame:
    # create a copy of the dataset
    video_vectors = dataset.copy()

    # get the embeddings for the query    
    query_embeddings = client.embeddings.create(input=query, model=model).data[0].embedding

    # create a new column with the calculated similarity for each row
    video_vectors["similarity"] = video_vectors["ada_v2"].apply(
        lambda x: cosine_similarity(np.array(query_embeddings), np.array(x))
    )

    # filter the videos by similarity
    mask = video_vectors["similarity"] >= SIMILARITIES_RESULTS_THRESHOLD
    video_vectors = video_vectors[mask].copy()

    # sort the videos by similarity
    video_vectors = video_vectors.sort_values(by="similarity", ascending=False).head(
        rows
    )

    # return the top rows
    return video_vectors.head(rows)

Diese Funktion ist sehr einfach, sie gibt nur die Ergebnisse der Suchanfrage aus.


In [ ]:
def display_results(videos: pd.core.frame.DataFrame, query: str):
    def _gen_yt_url(video_id: str, seconds: int) -> str:
        """convert time in format 00:00:00 to seconds"""
        return f"https://youtu.be/{video_id}?t={seconds}"

    print(f"\nVideos similar to '{query}':")
    for _, row in videos.iterrows():
        youtube_url = _gen_yt_url(row["videoId"], row["seconds"])
        print(f" - {row['title']}")
        print(f"   Summary: {' '.join(row['summary'].split()[:15])}...")
        print(f"   YouTube: {youtube_url}")
        print(f"   Similarity: {row['similarity']}")
        print(f"   Speakers: {row['speaker']}")

1. Zuerst wird der Embedding-Index in ein Pandas Dataframe geladen.
2. Anschließend werden Sie aufgefordert, eine Suchanfrage einzugeben.
3. Danach wird die Funktion `get_videos` aufgerufen, um im Embedding-Index nach der Anfrage zu suchen.
4. Schließlich wird die Funktion `display_results` aufgerufen, um die Ergebnisse für Sie anzuzeigen.
5. Danach werden Sie erneut aufgefordert, eine weitere Suchanfrage einzugeben. Dieser Vorgang wiederholt sich, bis Sie `exit` eingeben.

![](../../../../translated_images/notebook-search.1e320b9c7fcbb0bc1436d98ea6ee73b4b54ca47990a1c952b340a2cadf8ac1ca.de.png)

Sie werden aufgefordert, eine Suchanfrage einzugeben. Geben Sie eine Anfrage ein und drücken Sie die Eingabetaste. Die Anwendung gibt eine Liste von Videos zurück, die zur Anfrage passen. Außerdem erhalten Sie einen Link zu der Stelle im Video, an der die Antwort auf Ihre Frage zu finden ist.

Hier sind einige Beispielanfragen, die Sie ausprobieren können:

- Was ist Azure Machine Learning?
- Wie funktionieren Convolutional Neural Networks?
- Was ist ein neuronales Netzwerk?
- Kann ich Jupyter Notebooks mit Azure Machine Learning verwenden?
- Was ist ONNX?


In [ ]:
pd_vectors = load_dataset(DATASET_NAME)

# get user query from imput
while True:
    query = input("Enter a query: ")
    if query == "exit":
        break
    videos = get_videos(query, pd_vectors, 5)
    display_results(videos, query)


---

**Haftungsausschluss**:  
Dieses Dokument wurde mit dem KI-Übersetzungsdienst [Co-op Translator](https://github.com/Azure/co-op-translator) übersetzt. Obwohl wir uns um Genauigkeit bemühen, beachten Sie bitte, dass automatisierte Übersetzungen Fehler oder Ungenauigkeiten enthalten können. Das Originaldokument in seiner Ausgangssprache gilt als maßgebliche Quelle. Für wichtige Informationen wird eine professionelle menschliche Übersetzung empfohlen. Wir übernehmen keine Haftung für Missverständnisse oder Fehlinterpretationen, die sich aus der Nutzung dieser Übersetzung ergeben.
